### CityPOI Explorer: H3 Indexing with Google Places

Using Uber H3 spatial indexing to get city-wide POI data from **Google Places API** - developing a technique that leverages **Uber**'s H3 spatial indexing algorithm to extract Point of Interest (POI) data for large geographies / areas like cities from the Google Places API.

#### Setting up the Places API

In [ ]:
!pip install -U folium h3 tqdm ast geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests as re
import folium
import h3 
import tqdm
import requests
import json
import time
import os
import math
import ast

from shapely import geometry
from shapely.ops import unary_union, transform

pd.set_option('display.max_columns',None)

# Please do not keep the API key in the open like this in
# an actual experiment, please.
API_KEY = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Shapefiles will be provided in the github repository
districts_gpd = gpd.read_file('data/admin_boundaries/Adminbdy Shapefile/District_Boundary.shp')
karachi_districts = ['KARACHI CENTRAL', 'KARACHI WEST', 'MALIR CANTONMENT',
       'KORANGI CREEK CANTONMENT', 'MANORA CANTONMENT',
       'CLIFTON CANTONMENT', 'KARACHI CANTONMENT', 'FAISAL CANTONMENT',
       'KARACHI SOUTH', 'MALIR', 'KORANGI', 'KARACHI EAST']

# Unary union to combine all districts into one polygon object
khi_boundaries = unary_union(districts_gpd[districts_gpd\
  .DISTRICT\
  .isin(karachi_districts)==True]['geometry'])

# Small buffer to smoothen the edges
khi_boundaries = khi_boundaries.buffer(0.05)
khi_boundaries

In [ ]:
# Flip coordinate sequence from longitude, latitude 
# to latitude, longitude as this is how the h3 API reads it
def flip(x,y):
    return y,x
khi_boundaries = transform(flip,khi_boundaries)

# Convert to polgyon geojson object
khi_boundaries_gejson = gpd.GeoSeries([khi_boundaries])\
    .__geo_interface__['features'][0]['geometry']

# Get h3 cell IDs for cells within the bounding polygon / geojson
res6_khi_hex = h3.polyfill(khi_boundaries_gejson,6)

In [ ]:
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    original source: https://nbviewer.org/github/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb

    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # Flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium\
            .Map(location=[sum(lat)/len(lat),
                        sum(lng)/len(lng)],
                        zoom_start=9, 
                        tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=4,color=color)
        m.add_child(my_PolyLine)
    return m

m = visualize_hexagons(hexagons=res6_khi_hex)
display(m)

In [ ]:
def get_places_poi(lat,lng,resolution,type,api_key):
    places_df = []

    # nearby sarch API URL
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
    'location': f'{lat},{lng}',
    'radius': resolution,
    'types': type,
    'key': api_key}

    response = requests.get(url, params=params)
    results = json.loads(response.content)
    places_df.append(results['results'])

    while 'next_page_token' in results:
        time.sleep(1)
        params['pagetoken'] = results['next_page_token']
        response = requests.get(url,params=params)
        results = json.loads(response.content)
        places_df.append(results['results'])
        
    results_df = pd.concat([pd.DataFrame(df) for df in places_df])
    results_df = results_df.reset_index(drop=True)
    
    return results_df

# Location for Karachi DHA Phase 6, Khayaban-e-Sehar
lat,lng = 24.794386, 67.048928

# see all POI types for places API here: https://developers.google.com/maps/documentation/places/web-service/supported_types
results_df = get_places_poi(lat=lat,lng=lng,
                                 resolution=2000,
                                 type='cafe', # 
                                 api_key=API_KEY)

results_df[['place_id','name','types','user_ratings_total','vicinity']].head()

In [ ]:
res6_khi_hex = pd.DataFrame({'hex_id':list(res6_khi_hex),'hex_resolution':6})

res7_khi_hex = h3.polyfill(khi_boundaries_gejson,7)
res7_khi_hex = pd.DataFrame({'hex_id':list(res7_khi_hex),'hex_resolution':7})

res8_khi_hex = h3.polyfill(khi_boundaries_gejson,8)
res8_khi_hex = pd.DataFrame({'hex_id':list(res8_khi_hex),'hex_resolution':8})

res9_khi_hex = h3.polyfill(khi_boundaries_gejson,9)
res9_khi_hex = pd.DataFrame({'hex_id':list(res9_khi_hex),'hex_resolution':9})

khi_hex_df = pd.concat([res6_khi_hex,res7_khi_hex,res8_khi_hex,res9_khi_hex])
khi_hex_df.reset_index(drop=True,inplace=True)

def get_hex_radius(row,BUFFER):
    edge_length = h3.edge_length(row.hex_resolution,'m')
    radius = edge_length
    radius = radius+edge_length*BUFFER
    return radius

khi_hex_df['hex_area'] = khi_hex_df.hex_id.apply(lambda x: h3.cell_area(x,unit='m^2'))
khi_hex_df['hex_radius_places_api'] = khi_hex_df.apply(lambda row: get_hex_radius(row,0.15),axis=1)
khi_hex_df['centroid'] = khi_hex_df.hex_id.apply(lambda x: h3.h3_to_geo(x))
khi_hex_df.head()

In [ ]:
#res6 ~3700m
#Get the highest resolution cells
max_khi_hex_df = khi_hex_df[khi_hex_df.hex_resolution==khi_hex_df.hex_resolution.min()]

# We created a seperate folder to store all output, in case the code is interrupted we can use
# this list to make sure we're not repeating any cells.

searched_cells = os.listdir(f'data/h3cell_output_cafe/')
searched_cells = set([x.replace('.csv','') for x in searched_cells])

# Setting POI type for cafe.
type = 'cafe'

for ind,row in tqdm.tqdm(max_khi_hex_df.iterrows(),
                         total=max_khi_hex_df.shape[0]):
    # Parsing the highest resolution cells and saving outputs in a csv file
    h3_cell = row.hex_id
    if(h3_cell not in searched_cells):
        lat,lng = row.centroid[0],row.centroid[1]
        resolution = row.hex_radius_places_api
        # Call the Places using the the function we wrote earlier
        df = func_get_places_poi(lat=lat,lng=lng,resolution=resolution,type=type,api_key=API_KEY)
        df['hex_id'] = h3_cell
        cell_save_path = f'data/h3cell_output_{type}/{h3_cell}.csv'
        df.to_csv(cell_save_path,index=False)
        
        # Key part of the algorithm, if the API returns equal or more than 60 results
        if(len(df)>=60):
            cell_children = h3.h3_to_children(df['hex_id'][0])
            child_df = khi_hex_df[khi_hex_df.hex_id.isin(cell_children)]
            
            #res 7 ~1400m
            for ind,row in child_df.iterrows():
                h3_cell = row.hex_id
                lat,lng = row.centroid[0],row.centroid[1]
                resolution = row.hex_radius_places_api
                df = func_get_places_poi(lat=lat,lng=lng,resolution=resolution,type=type,api_key=API_KEY)
                df['hex_id'] = h3_cell
                cell_save_path = f'data/h3cell_output_{type}/{h3_cell}.csv'
                df.to_csv(cell_save_path,index=False)

                # Repeat same logic as above on all other resolutions  
                if(len(df)>=60):
                    cell_children = h3.h3_to_children(df['hex_id'][0])
                    child_df = khi_hex_df[khi_hex_df.hex_id.isin(cell_children)]

                    # res 8 ~500m
                    for ind,row in child_df.iterrows():
                        h3_cell = row.hex_id
                        lat,lng = row.centroid[0],row.centroid[1]
                        resolution = row.hex_radius_places_api
                        df = func_get_places_poi(lat=lat,lng=lng,resolution=resolution,type=type,api_key=API_KEY)
                        df['hex_id'] = h3_cell
                        cell_save_path = f'data/h3cell_output_{type}/{h3_cell}.csv'
                        df.to_csv(cell_save_path,index=False)

                        # You can go as deep as you want, down to res 9 and res 10 similary
                        # Yes, this should be a recursive funciton but I'll leave that to you? :)

In [ ]:
# We can see that the algorithm did search 34 lower resolution hexagons,
# where we managed to saturate the API at a higher reesolution.
paths = os.listdir('data/h3cell_output_cafe')
paths = ['data/h3cell_output_cafe/' + path for path in paths]
n_df = pd.concat([pd.read_csv(path) for path in paths])
khi_hex_df[khi_hex_df\
           .hex_id\
           .isin(n_df.hex_id.unique())]\
           .hex_resolution.value_counts()

In [ ]:
# Important we first de-duplicate our data by using the place_id,
# which is unique for each POI on Google Maps.
n_df = n_df.drop_duplicates(subset='place_id')
n_df = n_df.reset_index(drop=True)
n_df['location'] = n_df.geometry.apply(lambda x: ast.literal_eval(x)['location'])
n_df['lat'] = n_df['location'].apply(lambda x: x['lat'])
n_df['lng'] = n_df['location'].apply(lambda x: x['lng'])
for ind,row in n_df.iterrows():
    lat = row.lat
    lng = row.lng
    folium.CircleMarker(location=(lat,lng),
                        radius=5,
                        color='blue').add_to(m)

m